## 使用する仮想環境: "orthogonal" anaconda>envs>orthogonalを自分のPCの同じ場所にコピー

# 生画像 (hdf5) をorthogonal_viewに変換する

In [ ]:
import h5py
import numpy as np
import tifffile as tiff
import os
from binary2hdf5 import  read_virtual_hdf5


# HDF5ファイルの読み込み
hdf5_file = rhdf5_file = r"\\4-307-KM\c.elegans_raw\fluorescence_image\24-05-08\20240508-185610tdTomato-10mW-1 good\raw_10vps.h5"


output_path = r"\\4-307-KM\c.elegans_raw\fluorescence_image\24-05-08\20240508-185610tdTomato-10mW-1 good"
dirname = os.path.splitext(os.path.basename(hdf5_file))[0] 

if not os.path.exists(output_path):
    os.makedirs(output_path)

with h5py.File(hdf5_file, 'r') as file:
    array = file['default']
    print (array.shape)
    volumes = array.shape[0]
    z = array.shape[2]
    y = array.shape[3]
    x = array.shape[4]

orthogonal_view = np.zeros((volumes,2,y+z+3,x+z+3),dtype = 'uint16')

for channel in range(2):
    print(f"now proccesing ch={channel}...")
    

    for t in range(volumes):
        print(f"now proccesing t={t}...")
        data=read_virtual_hdf5(hdf5_path=hdf5_file, t=t+1)
        data=data[channel,...]

        
        
    # データが (t, x, y, z) の形状であると仮定

        orthogonal_view[t,channel,y+z+2:y+z+3,x+z+2:x+z+3] = 1000

        orthogonal_view[t,channel,0:y,0:x]=np.max(data, axis=0)
        orthogonal_view[t,channel,0:y,x+3:x+z+3]=np.transpose(np.max(data,axis=2))
        orthogonal_view[t,channel,y+3:y+z+3,0:x]=np.max(data,axis=1)

        # 直交ビューを作成する関数

        # 各時刻のフレームを生成し、動画として保存

        
file_name = f"orthogonal_view_{dirname}.tif"
output_path2 = os.path.join(output_path, file_name)
        #tiff.imsave(file_path, three_dimensional_image)
tiff.imwrite(output_path2, orthogonal_view,imagej=True, metadata={'axes': 'TCYX'}, compression ='zlib')



#蛍光とラベルをマージ, ラベル値が大きいものが手前に来る

In [ ]:
#蛍光とラベルをマージ, ラベル値が大きいものが手前に来る
import re
import os
import h5py
import numpy as np
import tifffile as tiff
from pathlib import Path
from matplotlib.colors import ListedColormap
from binary2hdf5 import read_virtual_hdf5  # 元コードと同じ関数を使用

# ========= ユーザー設定 =========
# 入力（ラベルTIFFフォルダ & 蛍光HDF5）
label_dir   = Path(r"\\4-307-KM\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish")
hdf5_file   = r"\\4-307-KM\c.elegans_raw\fluorescence_image\24-05-08\20240508-185610tdTomato-10mW-1 good\raw_10vps.h5"

# 出力（マージ結果RGB, 参照用の単独直交ビューも出せる）
out_dir     = Path(r"\\4-307-KM\c.elegans_raw\fluorescence_image\24-05-08\20240508-185610tdTomato-10mW-1 good")
out_dir.mkdir(parents=True, exist_ok=True)

# 対象 t（1始まり, 例: t000634）
target_t    = 634
# 対象ch（0 or 1）
channel     = 0

# 表示調整（蛍光→8bitグレースケール化のレンジ。Noneならパーセンタイルで決める）
fluor_vmin_percent = 1.0
fluor_vmax_percent = 99.0

# ラベル着色のα（0..1）
alpha = 0.6

# マージン幅（元コードは 3 px）
margin = 3
# ==============================


def build_orthogonal_fluorescence(hdf5_path, t_1based, ch, margin):
    """HDF5から (z,y,x) を取り出し、元コードと同じ配置・margin=3の直交ビュー(uint16)を返す。"""
    data = read_virtual_hdf5(hdf5_path=hdf5_path, t=t_1based)  # 期待形状: (channel, z, y, x)
    data = data[ch, ...]                                       # (z, y, x)

    z, y, x = data.shape
    canvas_h = y + z + margin
    canvas_w = x + z + margin

    ort = np.zeros((canvas_h, canvas_w), dtype=data.dtype)
    # 右下隅の1ピクセルマーカー（元コード相当）
    ort[y+z+2:y+z+3, x+z+2:x+z+3] = np.array(1000, dtype=data.dtype)

    # XY（Z方向max）
    ort[0:y, 0:x] = np.max(data, axis=0)
    # XZ（Y方向max → 転置で(y,z)）
    ort[0:y, x+margin:x+margin+z] = np.max(data, axis=2).T
    # YZ（X方向max）
    ort[y+margin:y+margin+z, 0:x] = np.max(data, axis=1)

    return ort, (y, x, z)


def load_label_stack_for_t(label_dir: Path, target_t: int):
    """label_dir 内から t000XXX を含むスライス群（z順）を (z,y,x) に積んで返す。"""
    t_tag = f"t{target_t:06}".lower()
    all_tiffs = list(label_dir.glob("*.tif*"))  # .tif / .tiff
    subset_t  = [p for p in all_tiffs if t_tag in p.name.lower()]
    if not subset_t:
        raise FileNotFoundError(f"'{t_tag}' を含むラベルTIFFが見つかりません: {label_dir}")

    # _z#### の番号を抽出（桁数揺れOK）。_z が無い場合は r"z(\d+)" に変更
    z_pat = re.compile(r"_z(\d+)", re.IGNORECASE)
    z_list = []
    for p in subset_t:
        m = z_pat.search(p.name)
        if m:
            z_idx = int(m.group(1))
            z_list.append((z_idx, p))
    if not z_list:
        head = [p.name for p in subset_t[:5]]
        raise FileNotFoundError(f"tは見つかったが _z#### が無い命名です。例: {head}")

    z_list.sort(key=lambda x: x[0])
    # スタック化
    sample = tiff.imread(z_list[0][1])
    y, x   = sample.shape
    dtype  = sample.dtype
    z      = len(z_list)

    stack = np.zeros((z, y, x), dtype=dtype)
    for i, (_, p) in enumerate(z_list):
        im = tiff.imread(p)
        if im.shape != (y, x):
            raise ValueError(f"サイズ不一致 {p}: {im.shape} != {(y, x)}")
        if im.dtype != dtype:
            im = im.astype(dtype, copy=False)
        stack[i] = im

    return stack  # (z,y,x)


def build_orthogonal_label(stack, margin):
    """(z,y,x) のラベルを元コードと同じ配置・margin=3で直交ビューに（数値ラベルのまま）"""
    z, y, x = stack.shape
    canvas_h = y + z + margin
    canvas_w = x + z + margin

    ort = np.zeros((canvas_h, canvas_w), dtype=stack.dtype)
    # 右下隅の1ピクセルマーカー（蛍光に合わせる）
    if np.issubdtype(ort.dtype, np.integer):
        marker_val = np.iinfo(ort.dtype).max // 65  # 適当な非ゼロ（蛍光の1000に意味合わせたいときは調整）
    else:
        marker_val = 1
    ort[y+z+2:y+z+3, x+z+2:x+z+3] = marker_val

    # XY（Z方向max）※元コードと同じく MIP(max)
    ort[0:y, 0:x] = np.max(stack, axis=0)
    # XZ（Y方向max → 転置で(y,z)）
    ort[0:y, x+margin:x+margin+z] = np.max(stack, axis=2).T
    # YZ（X方向max）
    ort[y+margin:y+margin+z, 0:x] = np.max(stack, axis=1)

    return ort


def make_margin_mask(y, x, z, margin, canvas_h, canvas_w):
    """交差も含め、マージン帯を True にするマスク"""
    mask = np.zeros((canvas_h, canvas_w), dtype=bool)
    y0, x0, m = y, x, margin
    # 横帯
    mask[y0:y0+m, 0:x0] = True
    mask[y0:y0+m, x0+m:x0+m+z] = True
    # 縦帯
    mask[0:y0, x0:x0+m] = True
    mask[y0+m:y0+m+z, x0:x0+m] = True
    # ★交差ブロック（中央）も白に
    mask[y0:y0+m, x0:x0+m] = True
    return mask


def to_uint8_rgb_from_uint16_gray(gray, vmin=None, vmax=None):
    """uint16灰度 → [0,255] uint8 3ch。vmin/vmaxがNoneならパーセンタイルで自動決定"""
    g = gray.astype(np.float32)
    if vmin is None or vmax is None:
        lo = np.percentile(g, fluor_vmin_percent)
        hi = np.percentile(g, fluor_vmax_percent)
        if hi <= lo:
            lo, hi = float(g.min()), float(g.max())
    else:
        lo, hi = float(vmin), float(vmax)
    if hi == lo:
        hi = lo + 1.0
    g = np.clip((g - lo) / (hi - lo), 0.0, 1.0)
    g8 = (g * 255.0 + 0.5).astype(np.uint8)
    rgb = np.stack([g8, g8, g8], axis=-1)
    return rgb


def colorize_labels_uint8(ort_label):
    """元コードと同じ 256色ランダムLUTで RGB へ（0→黒, seed=42）。ラベル>255ならエラーにします。"""
    vmax = int(ort_label.max())
    if vmax > 255:
        raise ValueError(f"ラベル最大値 {vmax} > 255。元コード互換の256色LUTでは不足です（再マップ必要）。")
    np.random.seed(42)
    lut = np.random.rand(256, 3)
    lut[0] = [0, 0, 0]
    rgb = (lut[ort_label] * 255).astype(np.uint8)  # (H,W,3)
    return rgb


def merge_label_on_fluor(f_rgb, l_rgb, l_mask, alpha=0.6):
    """labelマスクの場所だけ α ブレンドで重ねる"""
    # f_rgb, l_rgb: uint8 RGB, l_mask: bool (label>0)
    out = f_rgb.copy()
    # α合成： out = (1-α)*F + α*L
    fr = out.astype(np.float32)
    lr = l_rgb.astype(np.float32)
    fr[l_mask] = (1.0 - alpha) * fr[l_mask] + alpha * lr[l_mask]
    return fr.astype(np.uint8)


# ======== 実行 ========
# 1) 蛍光の直交ビュー
fluor_ort, (Y, X, Z) = build_orthogonal_fluorescence(hdf5_file, target_t, channel, margin)

# 2) ラベルの直交ビュー
label_stack = load_label_stack_for_t(label_dir, target_t)   # (z,y,x)
if label_stack.shape != (Z, Y, X):
    raise ValueError(f"ラベルと蛍光のサイズ不一致: label {label_stack.shape} vs fluor {(Z, Y, X)}")
label_ort = build_orthogonal_label(label_stack, margin)

# 3) マージンマスク（交差含む）
canvas_h = Y + Z + margin
canvas_w = X + Z + margin
margin_mask = make_margin_mask(Y, X, Z, margin, canvas_h, canvas_w)

# 4) 可視化用にRGB化
fluor_rgb = to_uint8_rgb_from_uint16_gray(fluor_ort)     # 蛍光→8bitグレイ→RGB
label_rgb = colorize_labels_uint8(label_ort)             # ラベル→LUTでRGB

# 5) ラベル位置のブレンド（label>0 のみ）
label_mask = (label_ort > 0)
merged = merge_label_on_fluor(fluor_rgb, label_rgb, label_mask, alpha=alpha)

# 6) マージンを純白に（交差も白）
merged[margin_mask] = 255

# 7) 右下隅の1ピクセルマーカーも白に（視覚的に統一）
merged[Y+Z+2:Y+Z+3, X+Z+2:X+Z+3, :] = 255

# 8) 保存
merged_path = out_dir / f"orthogonal_merge_t{target_t:06}_ch{channel}.tif"
tiff.imwrite(merged_path, merged, photometric='rgb')
print("[OK] Saved merged:", merged_path)

# 参考用に単独出力したい場合:
# tiff.imwrite(out_dir / f"orthogonal_fluor_t{target_t:06}_ch{channel}.tif", fluor_rgb, photometric='rgb')
# tiff.imwrite(out_dir / f"orthogonal_label_t{target_t:06}.tif", label_rgb, photometric='rgb')


[OK] Saved merged: \\4-307-KM\c.elegans_raw\fluorescence_image\24-05-08\20240508-185610tdTomato-10mW-1 good\orthogonal_merge_t000634_ch0.tif


###xy, xz, zyのラベル画像はz方向の手前のものは手前に、深いところにあるものは深いところにくるように

In [ ]:
###xy, xz, zyのラベル画像はz方向の手前のものは手前に、深いところにあるものは深いところにくるように
import re
import os
import h5py
import numpy as np
import tifffile as tiff
from pathlib import Path
from matplotlib.colors import ListedColormap
from binary2hdf5 import read_virtual_hdf5  # 元コードの関数

# ========= ユーザー設定 =========
# 入力（ラベルTIFFフォルダ & 蛍光HDF5）
label_dir   = Path(r"\\4-307-KM\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish")
hdf5_file   = r"\\4-307-KM\c.elegans_raw\fluorescence_image\24-05-08\20240508-185610tdTomato-10mW-1 good\raw_10vps.h5"

# 出力
out_dir     = Path(r"\\4-307-KM\c.elegans_raw\fluorescence_image\24-05-08\20240508-185610tdTomato-10mW-1 good")
out_dir.mkdir(parents=True, exist_ok=True)

# 対象 t（1始まり, 例: t000634）
target_t    = 634
# 対象ch（0 or 1）
channel     = 0

# 蛍光→8bit化のレンジ（パーセンタイル指定）
fluor_vmin_percent = 1.0
fluor_vmax_percent = 99.0

# ラベル着色のα（0..1）
alpha = 0.6

# マージン幅（元コードは 3 px）
margin = 3
# ==============================


def first_nonzero_along_axis(arr, axis: int):
    """arr!=0 となる最初の要素値を axis 方向で選ぶ。全ゼロは0。"""
    mask = (arr != 0)
    idx = mask.argmax(axis=axis)  # 最初にTrueになった位置（全Falseは0）
    picked = np.take_along_axis(arr, np.expand_dims(idx, axis), axis=axis).squeeze(axis=axis)
    return np.where(mask.any(axis=axis), picked, 0)


def build_orthogonal_fluorescence(hdf5_path, t_1based, ch, margin):
    """HDF5から (z,y,x) を取り出し、元コードと同じ配置・margin=3の直交ビュー(uint16)を返す。"""
    data = read_virtual_hdf5(hdf5_path=hdf5_path, t=t_1based)  # 期待: (channel, z, y, x)
    data = data[ch, ...]                                       # (z, y, x)

    z, y, x = data.shape
    canvas_h = y + z + margin
    canvas_w = x + z + margin

    ort = np.zeros((canvas_h, canvas_w), dtype=data.dtype)
    # 右下隅の1ピクセルマーカー（元コード相当）
    ort[y+z+2:y+z+3, x+z+2:x+z+3] = np.array(1000, dtype=data.dtype)

    # XY（Z方向max）
    ort[0:y, 0:x] = np.max(data, axis=0)
    # XZ（Y方向max → 転置で(y,z)）
    ort[0:y, x+margin:x+margin+z] = np.max(data, axis=2).T
    # YZ（X方向max）
    ort[y+margin:y+margin+z, 0:x] = np.max(data, axis=1)

    return ort, (y, x, z)


def load_label_stack_for_t(label_dir: Path, target_t: int):
    """label_dir 内から t000XXX を含むスライス群（z順）を (z,y,x) に積んで返す。"""
    t_tag = f"t{target_t:06}".lower()
    all_tiffs = list(label_dir.glob("*.tif*"))  # .tif / .tiff
    subset_t  = [p for p in all_tiffs if t_tag in p.name.lower()]
    if not subset_t:
        raise FileNotFoundError(f"'{t_tag}' を含むラベルTIFFが見つかりません: {label_dir}")

    # _z#### の番号を抽出（桁数揺れOK）。_z が無い場合は r"z(\d+)" に変更
    z_pat = re.compile(r"_z(\d+)", re.IGNORECASE)
    z_list = []
    for p in subset_t:
        m = z_pat.search(p.name)
        if m:
            z_idx = int(m.group(1))
            z_list.append((z_idx, p))
    if not z_list:
        head = [p.name for p in subset_t[:5]]
        raise FileNotFoundError(f"tは見つかったが _z#### が無い命名です。例: {head}")

    z_list.sort(key=lambda x: x[0])
    # スタック化
    sample = tiff.imread(z_list[0][1])
    y, x   = sample.shape
    dtype  = sample.dtype
    z      = len(z_list)

    stack = np.zeros((z, y, x), dtype=dtype)
    for i, (_, p) in enumerate(z_list):
        im = tiff.imread(p)
        if im.shape != (y, x):
            raise ValueError(f"サイズ不一致 {p}: {im.shape} != {(y, x)}")
        if im.dtype != dtype:
            im = im.astype(dtype, copy=False)
        stack[i] = im

    return stack  # (z,y,x)


def build_orthogonal_label_zpriority_all(stack, margin):
    """(z,y,x) のラベルを元コードと同じ配置・margin=3で直交ビューに。
       ★ XYは z 方向で first-nonzero（z優先）
       ★ XZは x 方向で first-nonzero（xを畳み込む面）
       ★ YZは y 方向で first-nonzero（yを畳み込む面）"""
    z, y, x = stack.shape
    canvas_h = y + z + margin
    canvas_w = x + z + margin

    ort = np.zeros((canvas_h, canvas_w), dtype=stack.dtype)
    # 右下隅の1ピクセルマーカー（蛍光に合わせる）
    if np.issubdtype(ort.dtype, np.integer):
        marker_val = np.iinfo(ort.dtype).max // 65
    else:
        marker_val = 1
    ort[y+z+2:y+z+3, x+z+2:x+z+3] = marker_val

    # --- 投影（ラベル値優先→手前優先に変更） ---
    # XY：z軸で“最初の非ゼロラベル”
    ort[0:y, 0:x] = first_nonzero_along_axis(stack, axis=0)       # (y,x)

    # XZ：x軸で“最初の非ゼロラベル” → (z,y) を作って転置で (y,z)
    xz = first_nonzero_along_axis(stack, axis=2)                  # (z,y)
    ort[0:y, x+margin:x+margin+z] = xz.T                          # (y,z)

    # YZ：y軸で“最初の非ゼロラベル” → (z,x)
    yz = first_nonzero_along_axis(stack, axis=1)                  # (z,x)
    ort[y+margin:y+margin+z, 0:x] = yz

    return ort


def make_margin_mask(y, x, z, margin, canvas_h, canvas_w):
    """交差も含め、マージン帯を True にするマスク"""
    mask = np.zeros((canvas_h, canvas_w), dtype=bool)
    y0, x0, m = y, x, margin
    # 横帯
    mask[y0:y0+m, 0:x0] = True
    mask[y0:y0+m, x0+m:x0+m+z] = True
    # 縦帯
    mask[0:y0, x0:x0+m] = True
    mask[y0+m:y0+m+z, x0:x0+m] = True
    # ★交差ブロック（中央）も白に
    mask[y0:y0+m, x0:x0+m] = True
    return mask


def to_uint8_rgb_from_uint16_gray(gray, vmin_percent=1.0, vmax_percent=99.0):
    """uint16灰度 → [0,255] uint8 3ch。パーセンタイルでレンジ決定"""
    g = gray.astype(np.float32)
    lo = np.percentile(g, vmin_percent)
    hi = np.percentile(g, vmax_percent)
    if hi <= lo:
        lo, hi = float(g.min()), float(g.max())
    if hi == lo:
        hi = lo + 1.0
    g = np.clip((g - lo) / (hi - lo), 0.0, 1.0)
    g8 = (g * 255.0 + 0.5).astype(np.uint8)
    rgb = np.stack([g8, g8, g8], axis=-1)
    return rgb


def colorize_labels_uint8(ort_label):
    """元コードと同じ 256色ランダムLUTで RGB へ（0→黒, seed=42）。ラベル>255ならエラー。"""
    vmax = int(ort_label.max())
    if vmax > 255:
        raise ValueError(f"ラベル最大値 {vmax} > 255。256色LUTでは不足（再マップ要）。")
    np.random.seed(42)
    lut = np.random.rand(256, 3)
    lut[0] = [0, 0, 0]
    rgb = (lut[ort_label] * 255).astype(np.uint8)  # (H,W,3)
    return rgb


def merge_label_on_fluor(f_rgb, l_rgb, l_mask, alpha=0.6):
    """labelマスクの場所だけ α ブレンドで重ねる"""
    out = f_rgb.copy()
    fr = out.astype(np.float32)
    lr = l_rgb.astype(np.float32)
    fr[l_mask] = (1.0 - alpha) * fr[l_mask] + alpha * lr[l_mask]
    return fr.astype(np.uint8)


# ======== 実行 ========
# 1) 蛍光の直交ビュー（元コードのmax投影のまま）
fluor_ort, (Y, X, Z) = build_orthogonal_fluorescence(hdf5_file, target_t, channel, margin)

# 2) ラベルの直交ビュー（★XYはz優先、XZはx優先(=first-nonzero)、YZはy優先(=first-nonzero)）
label_stack = load_label_stack_for_t(label_dir, target_t)   # (z,y,x)
if label_stack.shape != (Z, Y, X):
    raise ValueError(f"ラベルと蛍光のサイズ不一致: label {label_stack.shape} vs fluor {(Z, Y, X)}")
label_ort = build_orthogonal_label_zpriority_all(label_stack, margin)

# 3) マージンマスク（交差含む）
canvas_h = Y + Z + margin
canvas_w = X + Z + margin
margin_mask = make_margin_mask(Y, X, Z, margin, canvas_h, canvas_w)

# 4) 可視化用にRGB化
fluor_rgb = to_uint8_rgb_from_uint16_gray(fluor_ort, fluor_vmin_percent, fluor_vmax_percent)  # 蛍光→8bitグレイ→RGB
label_rgb = colorize_labels_uint8(label_ort)                                                  # ラベル→LUTでRGB

# 5) ラベル位置のブレンド（label>0 のみ）
label_mask = (label_ort > 0)
merged = merge_label_on_fluor(fluor_rgb, label_rgb, label_mask, alpha=alpha)

# 6) マージンを純白に（交差も白）
merged[margin_mask] = 255

# 7) 右下隅の1ピクセルマーカーも白に（視覚的に統一）
merged[Y+Z+2:Y+Z+3, X+Z+2:X+Z+3, :] = 255

# 8) 保存
merged_path = out_dir / f"orthogonal_merge_firstnonzero_allaxes_t{target_t:06}_ch{channel}.tif"
tiff.imwrite(merged_path, merged, photometric='rgb')
print("[OK] Saved merged:", merged_path)


[OK] Saved merged: \\4-307-KM\c.elegans_raw\fluorescence_image\24-05-08\20240508-185610tdTomato-10mW-1 good\orthogonal_merge_firstnonzero_allaxes_t000634_ch0.tif


# label (tiff)をorthogonal_viewに変換する

In [5]:
import numpy as np
import tifffile as tiff
import os

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap



In [ ]:
import numpy as np
import tifffile as tiff
import os

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


# 入力フォルダーと出力フォルダーの設定
folder_path = r"D:\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish"
output_path = r"D:\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish_ortho"

if not os.path.exists(output_path):
    os.makedirs(output_path)

x = 500
y = 300
z = 98
volumes = 1999

# tifファイルのリストを取得

tif_files = []

for filenames in os.listdir(folder_path):

    if filenames.endswith(".tif"):
        tif_files.append(os.path.join(folder_path, filenames))

        
for t in range(volumes):
    tif_files_t = []
    keyword = f"t{t+1:06}"

    for tif_file in tif_files:
        if keyword in tif_file:
            tif_files_t.append(tif_file)

    # 3次元画像の初期化
    three_dimentional_image = np.zeros((z, y, x), dtype='uint8')

    for zframe, tif_file in enumerate(tif_files_t):
        data = tiff.imread(tif_file)
        three_dimentional_image[zframe, :, :] = data

    print(three_dimentional_image.shape)

    # 直交ビューの作成

    orthogonal_view = np.zeros((y+z+3,x+z+3),dtype = 'uint8')
    orthogonal_view[y+z+2:y+z+3,x+z+2:x+z+3] = 255

    orthogonal_view[0:y,0:x]=np.max(three_dimentional_image, axis=0)
    orthogonal_view[0:y,x+3:x+z+3]=np.transpose(np.max(three_dimentional_image,axis=2))
    orthogonal_view[y+3:y+z+3,0:x]=np.max(three_dimentional_image,axis=1)
    orthogonal_view[650:660,680:802]= 255
    # 各時刻のフレームを画像として保存
    file_name = f"labels_orthogonal_t={t}.jpg"
    output_path2 = os.path.join(output_path, file_name)
# カラーマップを生成
    np.random.seed(42)  # 再現性のためにシードを固定
    colors = np.random.rand(256, 3)
    colors[0] = [0, 0, 0]  # 0の値は黒に設定
    cmap = ListedColormap(colors)

    # 画像を表示

    # 画像を保存
    plt.imsave(output_path2, orthogonal_view, cmap=cmap)

    print(f'Saved: {output_path2}')

(98, 300, 500)
Saved: D:\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish_ortho\labels_orthogonal_t=0.jpg
(98, 300, 500)
Saved: D:\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish_ortho\labels_orthogonal_t=1.jpg
(98, 300, 500)
Saved: D:\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish_ortho\labels_orthogonal_t=2.jpg
(98, 300, 500)
Saved: D:\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish_ortho\labels_orthogonal_t=3.jpg
(98, 300, 500)
Saved: D:\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track

In [3]:
import re
import numpy as np
import tifffile as tiff
from pathlib import Path

# === 入出力設定 ===
input_dir   = Path(r"\\4-307-KM\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish")   # ← Path(folder_path)
output_tiff = Path(r"\\4-307-KM\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish_ortho\ortho_t000634.tif")

# ★ ここでボリューム番号（1始まり）を指定
target_t = 634   # 例: t000517 を作りたい

# ---- 正規表現：指定tの z###### を拾う ----
# 例: ..._t000517_..._z000098.tif
pat = re.compile(rf"_t{target_t:06}.*_z(\d{{1,6}})\.tif$", re.IGNORECASE)

# === 指定tの zスライスを列挙し、zでソート ===
candidates = []
for p in input_dir.glob("*.tif"):
    m = pat.search(p.name)
    if m:
        z_idx = int(m.group(1))       # 1始まりの z 番号
        candidates.append((z_idx, p))

if not candidates:
    raise FileNotFoundError(f"t{target_t:06} の zスライスが見つかりません。パス/命名を確認してください。")

candidates.sort(key=lambda x: x[0])   # z 昇順

# === 以降は同じ（stack→直交ビュー→境界線→保存） ===
sample = tiff.imread(candidates[0][1])
y, x = sample.shape
dtype = sample.dtype
z = len(candidates)

stack = np.zeros((z, y, x), dtype=dtype)
for i, (_, path) in enumerate(candidates):
    img = tiff.imread(path)
    if img.shape != (y, x):
        raise ValueError(f"サイズ不一致: {path} {img.shape} != {(y, x)}")
    if img.dtype != dtype:
        img = img.astype(dtype, copy=False)
    stack[i] = img

margin = 5
canvas_h = y + z + margin
canvas_w = x + z + margin
orthogonal = np.zeros((canvas_h, canvas_w), dtype=dtype)

orthogonal[0:y, 0:x] = np.max(stack, axis=0)                       # XY
orthogonal[0:y, x+margin:x+margin+z] = np.max(stack, axis=2).T     # XZ
orthogonal[y+margin:y+margin+z, 0:x] = np.max(stack, axis=1)       # YZ

max_val = np.iinfo(dtype).max
orthogonal[y:y+margin, 0:x] = max_val
orthogonal[y:y+margin, x+margin:x+margin+z] = max_val
orthogonal[0:y, x:x+margin] = max_val
orthogonal[y+margin:y+margin+z, x:x+margin] = max_val

tiff.imwrite(output_tiff, orthogonal, imagej=True, metadata={"axes": "YX"})
print("Saved:", output_tiff)


Saved: \\4-307-KM\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish_ortho\ortho_t000634.tif


In [6]:
import re
import numpy as np
import tifffile as tiff
from pathlib import Path

# === 入出力設定 ===
input_dir   = Path(r"\\4-307-KM\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish")
output_rgb  = Path(r"\\4-307-KM\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish_ortho\ortho_t000634_RGB.tif")

target_t = 634  # 1始まり → t000634

# --- まず t番号を含むファイルだけ拾う（tif / tiff 両対応）---
t_tag = f"t{target_t:06}".lower()
all_tiffs = list(input_dir.glob("*.tif*"))
subset_t = [p for p in all_tiffs if t_tag in p.name.lower()]

print(f"all tif*: {len(all_tiffs)}  / files containing '{t_tag}': {len(subset_t)}")
if not subset_t:
    raise FileNotFoundError(f"'{t_tag}' を含むファイルが見つかりません。t番号が正しいか確認してください。")

# --- そこから _z???? を抜いて z昇順に並べる ---
z_pat = re.compile(r"_z(\d+)", re.IGNORECASE)  # 桁数は柔軟に
candidates = []
for p in subset_t:
    m = z_pat.search(p.name)
    if m:
        z_idx = int(m.group(1))      # 例: z0001 → 1
        candidates.append((z_idx, p))

print(f"matched z-slices: {len(candidates)}  (first 5: {[c[1].name for c in sorted(candidates)[:5]]})")
if not candidates:
    # デバッグ用に先頭5件のファイル名を出す
    head = [p.name for p in subset_t[:5]]
    raise FileNotFoundError(
        "t番号は見つかったが _z#### が見つかりませんでした。命名を確認してください。\n"
        f"例: {head}"
    )

candidates.sort(key=lambda x: x[0])

# === 3Dスタック作成 ===
sample = tiff.imread(candidates[0][1])
y, x = sample.shape
dtype = sample.dtype
z = len(candidates)

stack = np.zeros((z, y, x), dtype=dtype)
for i, (_, path) in enumerate(candidates):
    img = tiff.imread(path)
    if img.shape != (y, x):
        raise ValueError(f"サイズ不一致: {path} {img.shape} != {(y, x)}")
    if img.dtype != dtype:
        img = img.astype(dtype, copy=False)
    stack[i] = img

# === 直交ビュー（MIP） + 白マージン ===
margin = 5
canvas_h = y + z + margin
canvas_w = x + z + margin
orthogonal = np.zeros((canvas_h, canvas_w), dtype=dtype)

# XY
orthogonal[0:y, 0:x] = np.max(stack, axis=0)
# XZ
orthogonal[0:y, x+margin:x+margin+z] = np.max(stack, axis=2).T
# YZ
orthogonal[y+margin:y+margin+z, 0:x] = np.max(stack, axis=1)

# 白マージン位置（あとでRGBで上書き）
y0, x0, m = y, x, margin
margin_slices = [
    (slice(y0, y0+m),        slice(0, x0)),          # 横線（XY–YZ）
    (slice(y0, y0+m),        slice(x0+m, x0+m+z)),   # 横線（XY–XZ）
    (slice(0, y0),           slice(x0, x0+m)),       # 縦線（XY–XZ）
    (slice(y0+m, y0+m+z),    slice(x0, x0+m)),       # 縦線（YZ）
]

# === 元コードと同じ 256色ランダムLUT でRGB化 ===
if orthogonal.max() > 255:
    raise ValueError(f"ラベル最大値 {int(orthogonal.max())} > 255。"
                     "256色LUTでは足りません。ラベル再マップ等が必要です。")

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

np.random.seed(42)
colors = np.random.rand(256, 3)
colors[0] = [0, 0, 0]
cmap = ListedColormap(colors)

rgb = (colors[orthogonal] * 255).astype(np.uint8)

# マージンを純白に
for rs, cs in margin_slices:
    rgb[rs, cs, :] = 255

tiff.imwrite(output_rgb, rgb, photometric='rgb')
print("Saved RGB:", output_rgb)


all tif*: 195902  / files containing 't000634': 98
matched z-slices: 98  (first 5: ['track_results_t000634_z0001.tif', 'track_results_t000634_z0002.tif', 'track_results_t000634_z0003.tif', 'track_results_t000634_z0004.tif', 'track_results_t000634_z0005.tif'])
Saved RGB: \\4-307-KM\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish_ortho\ortho_t000634_RGB.tif


#マージンをすべて白ピクセル

In [8]:
import re
import numpy as np
import tifffile as tiff
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# === 入出力設定 ===
input_dir   = Path(r"\\4-307-KM\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish")
output_rgb  = Path(r"\\4-307-KM\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish_ortho\ortho_t000634_RGB.tif")
# 数値ラベル直交ビューも保存したい場合はパスを入れる（不要なら None）
output_label = None  # 例: Path(..., "ortho_t000634_LABEL.tif")

# ★ ボリューム番号（1始まり, 例: t000634）
target_t = 634

# === まず t を含むファイルに絞り込み（tif / tiff 両対応） ===
t_tag = f"t{target_t:06}".lower()
all_tiffs = list(input_dir.glob("*.tif*"))      # .tif / .tiff
subset_t  = [p for p in all_tiffs if t_tag in p.name.lower()]

print(f"[INFO] all tif*: {len(all_tiffs)} / contains '{t_tag}': {len(subset_t)}")
if not subset_t:
    raise FileNotFoundError(f"'{t_tag}' を含むファイルが見つかりません。t番号を確認してください。")

# === subset_t から z を抽出して昇順に ===
# 例: ..._z0001.tiff, ..._z0098.tif など（アンダースコア有り前提）
z_pat = re.compile(r"_z(\d+)", re.IGNORECASE)
candidates = []
for p in subset_t:
    m = z_pat.search(p.name)
    if m:
        z_idx = int(m.group(1))
        candidates.append((z_idx, p))

print(f"[INFO] matched z-slices: {len(candidates)}")
if not candidates:
    # 命名が _z でない場合は r"z(\d+)" に変更
    head = [p.name for p in subset_t[:5]]
    raise FileNotFoundError("t番号は見つかったが '_z####' が見つかりません。命名を確認してください。\n"
                            f"例: {head}")

candidates.sort(key=lambda x: x[0])

# === 3Dスタック作成 ===
sample = tiff.imread(candidates[0][1])
y, x = sample.shape
dtype = sample.dtype
z = len(candidates)

stack = np.zeros((z, y, x), dtype=dtype)
for i, (_, path) in enumerate(candidates):
    img = tiff.imread(path)
    if img.shape != (y, x):
        raise ValueError(f"サイズ不一致: {path} {img.shape} != {(y, x)}")
    if img.dtype != dtype:
        img = img.astype(dtype, copy=False)
    stack[i] = img

# === 直交ビュー（MIP: max） ===
margin = 3  # 元コード相当（3 px）
canvas_h = y + z + margin
canvas_w = x + z + margin
orthogonal = np.zeros((canvas_h, canvas_w), dtype=dtype)

# XY（Z方向max）
orthogonal[0:y, 0:x] = np.max(stack, axis=0)
# XZ（Y方向max → 転置して(y,z)に）
orthogonal[0:y, x+margin:x+margin+z] = np.max(stack, axis=2).T
# YZ（X方向max）
orthogonal[y+margin:y+margin+z, 0:x] = np.max(stack, axis=1)

# === マージンを一括で白にするためのマスク作成（交差部も確実に白） ===
y0, x0, m = y, x, margin
mask = np.zeros((canvas_h, canvas_w), dtype=bool)
# 横線（XY–YZ, XZ側）
mask[y0:y0+m, 0:x0] = True
mask[y0:y0+m, x0+m:x0+m+z] = True
# 縦線（XY–XZ, YZ側）
mask[0:y0, x0:x0+m] = True
mask[y0+m:y0+m+z, x0:x0+m] = True
mask[y0:y0+m, x0:x0+m] = True
# === 可視化：元コードと同じ 256色ランダムLUT で RGB 化 ===
if orthogonal.max() > 255:
    raise ValueError(f"ラベル最大値 {int(orthogonal.max())} > 255。"
                     "元コード互換の256色LUTでは不足します。ラベル再マップ等が必要です。")

np.random.seed(42)
colors = np.random.rand(256, 3)
colors[0] = [0, 0, 0]  # 背景0は黒
cmap = ListedColormap(colors)

rgb = (colors[orthogonal] * 255).astype(np.uint8)  # (H, W, 3)

# === マージン領域を“純白”で一括上書き（交差部も白） ===
rgb[mask] = 255

# === 保存 ===
tiff.imwrite(output_rgb, rgb, photometric='rgb')
print("[OK] Saved RGB:", output_rgb)

if output_label is not None:
    tiff.imwrite(output_label, orthogonal, imagej=True, metadata={"axes": "YX"})
    print("[OK] Saved LABEL:", output_label)


[INFO] all tif*: 195902 / contains 't000634': 98
[INFO] matched z-slices: 98
[OK] Saved RGB: \\4-307-KM\3DeeCellTracker\3DeeCellTracker_result\ultra_fast_light_sheet\240508\20240508-185610tdTomato-10mW-1 good\raw_ensemble\track_results\labels_corrected_finish_ortho\ortho_t000634_RGB.tif


## 豊島先生データのtiffファイルからorthogonal_viewを作成する

In [ ]:
import numpy as np
import tifffile as tiff
import os
from scipy import interpolate
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [ ]:
def stretch_array(arr, factor):
    original_rows, original_cols = arr.shape
    
    # 新しい行数を計算
    new_rows = original_rows * factor
    
    # 線形補間のためのx座標を設定
    x_old = np.linspace(0, original_rows - 1, original_rows)
    x_new = np.linspace(0, original_rows - 1, new_rows)
    
    # 各行の補間
    stretched_arr = np.zeros((new_rows, original_cols), dtype=arr.dtype)
    
    for col in range(original_cols):
        f = interpolate.interp1d(x_old, arr[:, col], kind='linear', fill_value='extrapolate')
        stretched_arr[:, col] = f(x_new)

    return stretched_arr



In [ ]:
folder_path = r"P:\Ascidiacea\orig\241025ホヤ\20220223-170215ASAPaiviatif_sequence"
output_path = r"P:\Ascidiacea\orig\241025ホヤ\20220223-170215ASAPaiviatif_sequence_orthogonal"

if not os.path.exists(output_path):
    os.makedirs(output_path)

x = 231
y = 297
z = 16
volumes = 2992

# tifファイルのリストを取得

tif_files = []

for filenames in os.listdir(folder_path):

    if filenames.endswith(".tif"):
        tif_files.append(os.path.join(folder_path, filenames))

        
for t in range(volumes):
    tif_files_t = []
    keyword = f"t{t+1:04}"

    for tif_file in tif_files:
        if keyword in tif_file:
            tif_files_t.append(tif_file)

    # 3次元画像の初期化
    three_dimentional_image = np.zeros((z, y, x), dtype='uint16')

    for zframe, tif_file in enumerate(tif_files_t):
        data = tiff.imread(tif_file)
        three_dimentional_image[zframe, :, :] = data

    print(three_dimentional_image.shape)

    # 直交ビューの作成

    stretched_z = 5 * z
    orthogonal_view = np.zeros((y + stretched_z + 3, x + stretched_z + 3), dtype='uint16')
    orthogonal_view[y + stretched_z + 2:y + stretched_z + 3, x + stretched_z + 2:x + stretched_z + 3] = 1000

    # 各軸の最大値を計算
    max_xy = np.max(three_dimentional_image, axis=0)
    max_xz = np.max(three_dimentional_image, axis=1)
    max_yz = np.max(three_dimentional_image, axis=2)

    max_yz=stretch_array(max_yz,5)
    max_xz=stretch_array(max_xz,5)
    print(max_xz.shape)
    print(max_yz.shape)


    orthogonal_view[0:y, 0:x] = max_xy
    orthogonal_view[0:y, x + 3:x + stretched_z + 3] = np.transpose(max_yz)
    orthogonal_view[y + 3:y + stretched_z + 3, 0:x] = max_xz

 





    file_name = f"image_raw_t={t}.tif"
    output_path2 = os.path.join(output_path, file_name)
    tiff.imwrite(output_path2, orthogonal_view,imagej=True, metadata={'axes': 'YX'}, compression ='zlib')




    print(f'Saved: {output_path2}')


## label動画用のorthogonal_view

In [ ]:
import numpy as np
import tifffile as tiff
import os
from scipy import interpolate
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [ ]:
def stretch_array(arr, factor):
    # 元の配列の形状を取得
    original_rows, original_cols = arr.shape
    
    # 新しい配列を初期化
    stretched_arr = np.zeros((original_rows * factor, original_cols), dtype=arr.dtype)
    
    # 各行を繰り返して新しい配列を作成
    for i in range(original_rows):
        stretched_arr[i * factor:(i + 1) * factor, :] = arr[i, :]

    return stretched_arr

In [ ]:
folder_path = r"P:\Ascidiacea\orig\241025ホヤ\20220223-170215ASAPaiviatif_label_sequence"
output_path = r"P:\Ascidiacea\orig\241025ホヤ\20220223-170215ASAPaiviatif_label_sequence_orthogonal"

if not os.path.exists(output_path):
    os.makedirs(output_path)

x = 231
y = 297
z = 16
volumes = 2991

# tifファイルのリストを取得

tif_files = []

for filenames in os.listdir(folder_path):

    if filenames.endswith(".tif"):
        tif_files.append(os.path.join(folder_path, filenames))

        
for t in range(volumes):
    tif_files_t = []
    keyword = f"t{t+1:04}"

    for tif_file in tif_files:
        if keyword in tif_file:
            tif_files_t.append(tif_file)

    # 3次元画像の初期化
    three_dimentional_image = np.zeros((z, y, x), dtype='uint8')

    for zframe, tif_file in enumerate(tif_files_t):
        data = tiff.imread(tif_file)
        three_dimentional_image[zframe, :, :] = data

    print(three_dimentional_image.shape)

    # 直交ビューの作成

    stretched_z = 5 * z
    orthogonal_view = np.zeros((y + stretched_z + 3, x + stretched_z + 3), dtype='uint8')
    orthogonal_view[y + stretched_z + 2:y + stretched_z + 3, x + stretched_z + 2:x + stretched_z + 3] = 1000

    # 各軸の最大値を計算
    max_xy = np.max(three_dimentional_image, axis=0)
    max_xz = np.max(three_dimentional_image, axis=1)
    max_yz = np.max(three_dimentional_image, axis=2)

    max_yz=stretch_array(max_yz,5)
    max_xz=stretch_array(max_xz,5)
    print(max_xz.shape)
    print(max_yz.shape)


    orthogonal_view[0:y, 0:x] = max_xy
    orthogonal_view[0:y, x + 3:x + stretched_z + 3] = np.transpose(max_yz)
    orthogonal_view[y + 3:y + stretched_z + 3, 0:x] = max_xz

 





    file_name = f"image_label_t={t}.jpg"
    output_path2 = os.path.join(output_path, file_name)



# カラーマップを生成
    # 画像を表示
    np.random.seed(42)  # 再現性のためにシードを固定
    colors = np.random.rand(256, 3)
    colors[0] = [0, 0, 0]  # 0の値は黒に設定
    cmap = ListedColormap(colors)

    # 画像を表示

    # 画像を保存
    plt.imsave(output_path2, orthogonal_view, cmap=cmap)
    # 画像を保存

    print(f'Saved: {output_path2}')